##1 Descarga de librerias

In [1]:
!pip install tpot

In [2]:
!python -m pip install dask distributed --upgrade


Requirement already up-to-date: dask in /Users/fernando/miniconda3/lib/python3.8/site-packages (2020.12.0)
Requirement already up-to-date: distributed in /Users/fernando/miniconda3/lib/python3.8/site-packages (2020.12.0)


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.model_selection import train_test_split

import time
import tpot # Added library
from tpot import TPOTClassifier

## Data, original, groups

### Reading kaggel data

Github might be full if dowloading from it


In [ ]:
# How to mount kaggle api in colab: 
# https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a 
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
#Check the present working directory using pwd command

In [ ]:
!kaggle datasets download -d mlg-ulb/creditcardfraud
!ls

In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

In [ ]:
#Reading the file
df= pd.read_csv('/content/gdrive/My Drive/Kaggle/creditcard.csv')
df.shape

## Reading from own linkry


In [4]:
#df = pd.read_csv('/content/creditcard.csv') # Emilio's link
df = pd.read_csv('~/Desktop/AutoML/creditcard.csv') # Fernando's link
df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [5]:
# In order to compare amounts regarding their relative sizes
#display(df['Amount'].isnull().sum()) # none
df['Amount']= (np.log(df['Amount'])).replace([np.inf, -np.inf], np.nan)

fraudulent_data = df.loc[df['Class'] == 1]
nonfraudulent_data = df.loc[df['Class'] == 0]

/Users/fernando/miniconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


#2. Exploración de los datos

In [ ]:
print('Groups and types',
      f'shape: {df.shape}',
      f'dtypes_aggregate: {df.dtypes.value_counts()}',
      f'fraud cases: {df.Class.sum()}',
      f'fraud cases (rate): {df.Class.sum()/df.shape[0]*100}',
      )

# There no duplicates
display(df.shape)
df.drop_duplicates()
display(df.shape)

print('How the infinites of log of Amount was distributed between groups')
display(df['Amount'].isna().sum(),
        fraudulent_data['Amount'].isna().sum(),
        nonfraudulent_data['Amount'].isna().sum())

display(df.head(3))
display(df.tail(3))


In [ ]:
display(df.describe(),
        fraudulent_data.describe(), nonfraudulent_data.describe())

In [ ]:
# Multimodal (more modes for non-fraudulent data) and few highly values 
plt.figure(figsize=(18,6))
#sns.kdeplot(df['Amount'], shade=True, color="r")
sns.kdeplot(fraudulent_data['Amount'], shade=True, color="r")
sns.kdeplot(nonfraudulent_data['Amount'], shade=True, color="b")

plt.show()

In [ ]:
plt.figure(figsize=(16,10))
plot = sns.distplot(a=df["Time"], kde=True, color='brown')
plot = sns.distplot(a=fraudulent_data["Time"], kde=True, color='purple')
plot = sns.distplot(a=nonfraudulent_data["Time"], kde=True, color='blue')

plot.set(xlabel ='Time', ylabel ='Frequency')
plt.show()

In [ ]:
# Correlation between groups differs sharply
df_corr = {}
df_corr["correlation"] =abs(df.corr()['Amount']).sort_values(ascending=False)
df_corr["correlation_fraud"] =abs(fraudulent_data.corr()['Amount']).sort_values(ascending=False)
df_corr["correlation_nonfraud"] =abs(nonfraudulent_data.corr()['Amount']).sort_values(ascending=False)
display(df_corr)
df_corr= pd.DataFrame(df_corr)
df_corr

In [ ]:
# Fraud variables more correlated than nonfraud variables

f,(ax1,ax2,ax3, axcb) = plt.subplots(1,4, 
            gridspec_kw={'width_ratios':[1,1,1,0.08]}, figsize=(18,6))
ax1.get_shared_y_axes().join(ax2,ax3)
g1 = sns.heatmap(df.corr(),cmap="YlGnBu",cbar=False,ax=ax1)
g1.set_ylabel('')
g1.set_xlabel('All')
g2 = sns.heatmap(fraudulent_data.corr(),cmap="YlGnBu",cbar=False,ax=ax2)
g2.set_ylabel('')
g2.set_xlabel('Fraud')
g2.set_yticks([])
g3 = sns.heatmap(nonfraudulent_data.corr(),cmap="YlGnBu",ax=ax3, cbar_ax=axcb)
g3.set_ylabel('')
g3.set_xlabel('Non-Fraud')
g3.set_yticks([])

In [ ]:

plt.figure(figsize=(18,12))
sns.scatterplot(x=df["Time"], y=df["Amount"], hue=df["Class"], size=df["Class"], sizes=(40, 8), marker="+")
# day night up down

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(x=fraudulent_data["Time"], y=fraudulent_data["Amount"], sizes=(40, 8), marker="+")
sns.scatterplot(x=nonfraudulent_data["Time"], y=nonfraudulent_data["Amount"], sizes=(40, 8), marker="+")


In [ ]:
plt.boxplot ([df['V1'], df['V2'], df['V3'], df['V4'], df['V5'], df['V6'], df['V7'], df['V8'], df['V9'], df['V10']])

In [ ]:
plt.boxplot ([df['V11'], df['V12'], df['V13'], df['V14'], df['V15'], df['V16'], df['V17'], df['V18'], df['V19'], df['V20']])


In [ ]:
plt.boxplot ([df['V21'], df['V22'], df['V23'], df['V24'], df['V25'], df['V26'], df['V27'], df['V28']])

# Parametrización de TPOT

TPOT cuenta con parámetros propios de los algoritmos genéticos. Estos resultan clave para la determinación del tiempo necesario para encontrar el pipeline que mejor ajuste al problema, como asi también la convergencia a una solución global.



## Parámetros de los algoritmos genéticos.

* **generations**: Es el número de iteraciones que repite el proceso de optimización del pipeline. El valor por defecto es de 100.



* **population_size:** int, optional (default=100)
Número de 'individuos' (soluciones) que se retienen de la poblacion de cada generación en la que corre el proceso.
Generally, TPOT will work better when you give it more individuals with which to optimize the pipeline.

* **offspring_size:** int, optional (default=None)
Número de 'offspring' (crías) que se producen en cada generación. Por defecto, es igual al numero de *population_size*.

* **mutation_rate:** float, optional (default=0.9)
Indica a que proporción de pipelines se le aplica modificaciones aleatorias en cada generación.

mutation_rate + crossover_rate no puede exceder 1.0.


* **crossover_rate:** float, optional (default=0.1)
Crossover rate for the genetic programming algorithm in the range [0.0, 1.0]. Indica qué proporción de pipelines se 'crían' en cada generación.

mutation_rate + crossover_rate cannot exceed 1.0.

Se recomienda dejar los valores por defecto si no se conocen los algoritmos genéticos.

La cantidad de pipelines va a estar determinada por: population_size + (generations * offspring_size).

En este caso establecemos 5 generaciones y dejamos el resto de las valiables en sus valores por defecto, por lo que tpot evaluará 600 pipelines

## Algunos otros parámetros de TPOT

* **scoring** : métrica utilizada para evaluar el ajuste del pipeline. En nuetro caso utilizamos 'f1' ya que trabajamos con un dataset altamente desbalanceado y de clasificación binaria.

* **cv**: Esquema de cross-validation para evaluar los pipelines. Dejamos el valor por defecto (5 carpetas).

* **subsamle** : Fracción de datos que se entrena en cada pipeline. Dejamos el valor por defecto (1.0).

* **max_time_mins**: Indica en cuántos minutos TPOT debe optimizar el pipeline. No establecemos ningún valor, permitiendo que TPOT corra el tiempo necesario.

* **max_eval_time_mins**: Indica el tiempo que tiene TPOT para evaluar un pipeline. Previene grandes pérdidas de tiempo, con la desventaja de dejar de lado pipelines mas complejos. Dejamos el valor por defecto de 5 minutos.



In [6]:
#Separamos la variable objetivo del dataset
df = df.dropna()
target = df.Class
df = df.drop(['Class'], axis=1)
display(df.isna().sum())


Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
dtype: int64

In [7]:
#Generamos los set de entrenamiento y test.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df, target, stratify=target)

In [8]:
#Generamos el pipeline

tpot = TPOTClassifier(generations=5, population_size=100,
                          offspring_size=None, mutation_rate=0.9,
                          crossover_rate=0.1,
                          scoring='f1', cv=5,
                          subsample=1.0, n_jobs=1,
                          max_time_mins=None, max_eval_time_mins=5,
                          random_state=43, config_dict=None,
                          warm_start=False,
                          memory=None,
                          periodic_checkpoint_folder=None,
                          early_stop=None,
                          verbosity=3,
                          disable_update_check=False)

In [ ]:

#Generamos la instancia de entrenamiento
start = time.time()

tpot.fit(x_train, y_train)

end = time.time()
print(f"Runtime of the program is {(end - start/60)} minutes")


32 operators have been imported by TPOT.


Skipped pipeline #10 due to time out. Continuing to the next pipeline.
Skipped pipeline #15 due to time out. Continuing to the next pipeline.
Skipped pipeline #18 due to time out. Continuing to the next pipeline.
Skipped pipeline #29 due to time out. Continuing to the next pipeline.
Skipped pipeline #31 due to time out. Continuing to the next pipeline.
Skipped pipeline #35 due to time out. Continuing to the next pipeline.
Skipped pipeline #37 due to time out. Continuing to the next pipeline.
Skipped pipeline #40 due to time out. Continuing to the next pipeline.
Skipped pipeline #42 due to time out. Continuing to the next pipeline.
Skipped pipeline #46 due to time out. Continuing to the next pipeline.
Skipped pipeline #53 due to time out. Continuing to the next pipeline.
Skipped pipeline #65 due to time out. Continuing to the next pipeline.
Skipped pipeline #70 due to time out. Continuing to the next pipeline.
Skipped pipeline #73 due to time out. Continuing to the next pipeline.
Skippe

_pre_test decorator: _random_mutation_operator: num_test=1 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=False.
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 [22:59:06] /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:567: Check failed: mparam_.num_feature != 0 (0 vs. 0) : 0 feature is supplied.  Are you using raw Booster interface?
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001253e923e dmlc::LogMessageFatal::~LogMessageFa

In [ ]:
#Generamos la instancia de predicciones
pred=tpot.predict(df)

In [ ]:
#Evaluamos el desempeño
tpot.score(x_test, y_test)


In [ ]:
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report, plot_precision_recall_curve, average_precision_score
print(confusion_matrix(target, pred))

In [ ]:
print(classification_report(target, pred))

In [ ]:
#Precision Recall curve
average_precision = average_precision_score(y_test, y_score)
disp = plot_precision_recall_curve(tpot, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))